<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part2/03_Procesamiento_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de Texto con LangChain 1.0 + Gemini
.

## A. Text Splitter.

* Divide texto (string)
* Trabaja sobre texto plano.
  - Recibe un str
  - Devuelve una lista de str
  - Aplica reglas de fragmentación

## 0. Instalación de librerías (Colab)

In [ ]:
!pip install -qU langchain  langchain-community langchain-google-genai google-generativeai langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 1. Carga del archivo

Subir archivo `Historia_Colombia.txt` al entorno de ejecución

In [ ]:
with open('Historia_Colombia.txt', encoding="utf8") as file:
    texto_completo = file.read()

## 2. Número de caracteres

In [ ]:
len(texto_completo)

159419

## 3. Número de palabras

In [ ]:
len(texto_completo.split())

22560

## 4. Transformador Character Text Splitter (LangChain 1.0)

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
)

texts = text_splitter.create_documents([texto_completo])

texts[:2]

[Document(metadata={}, page_content='HISTORIA DE COLOMBIA\nINTRODUCCIÓN\nLa historia de Colombia es un proceso complejo que se extiende desde los\nprimeros asentamientos humanos hasta la actualidad. El territorio\ncolombiano ha sido escenario de profundas transformaciones sociales,\npolíticas, económicas y culturales. Desde las civilizaciones indígenas\nhasta el Estado contemporáneo, Colombia ha construido una identidad\ndiversa y dinámica.\nPERÍODO PREHISPÁNICO\nMucho antes de la llegada de los europeos, el territorio estaba habitado\npor múltiples culturas indígenas. Entre las más destacadas se encuentran\nlos muiscas, taironas, quimbayas, zenúes y calimas. Estas sociedades\ndesarrollaron sistemas agrícolas avanzados, redes comerciales y\ncomplejas estructuras políticas.\nLos muiscas ocuparon el altiplano cundiboyacense y organizaron su\nsociedad en cacicazgos. La economía se basaba en el trueque, la sal y la\norfebrería. Los taironas construyeron asentamientos como Ciudad Perdida,\n

## 5. Longitud del primer fragmento

In [ ]:
len(texts[0].page_content)

1000

## 6. Segundo fragmento

In [ ]:
texts[1]

Document(metadata={}, page_content='En 1499 comenzaron las exploraciones europeas en las costas del actual\nterritorio colombiano. Durante el siglo XVI se consolidó el dominio\nespañol. Se fundaron ciudades como Santa Marta, Cartagena y Santafé de\nBogotá. El proceso implicó violencia, explotación y transformación\ncultural profunda.\nVIRREINATO DE LA NUEVA GRANADA\nEn 1717 se creó el Virreinato de la Nueva Granada. Se consolidó una\nsociedad jerarquizada y profundamente religiosa. Las reformas borbónicas\ngeneraron tensiones económicas y sociales.\nINDEPENDENCIA\nEl 20 de julio de 1810 inició el proceso independentista. Tras varios\naños de conflicto, la Batalla de Boyacá en 1819 aseguró la victoria\npatriota. Se creó la Gran Colombia, que posteriormente se disolvió en\n1830.\nSIGLO XIX\nEl siglo XIX estuvo marcado por guerras civiles entre liberales y\nconservadores. En 1886 se promulgó una nueva Constitución centralista.\nLa Guerra de los Mil Días devastó el país y precedió la separ

## 7. Longitud del segundo fragmento

In [ ]:
len(texts[1].page_content)

980

## 8. `RecursiveCharacterTextSplitter`

`CharacterTextSplitter` depende de un separador fijo (por ejemplo "\n" o ". "). Si el texto viene como un solo bloque largo sin muchos saltos de línea o sin el separador indicado, el splitter no encuentra dónde cortar correctamente y termina generando fragmentos más grandes que el chunk_size, produciendo advertencias y cortes poco naturales, como se vió en el ejemplo anterior.

`RecursiveCharacterTextSplitter` es mejor para este caso porque intenta dividir el texto usando varios separadores en orden (párrafos, líneas, oraciones, espacios) hasta encontrar una forma adecuada de ajustarse al tamaño definido. Eso lo hace más flexible y robusto cuando trabajamos con texto real que no siempre está bien formateado.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separators=["\n\n", "\n", ". ", " ", ""],  # va probando de más “natural” a más agresivo
)

texts = text_splitter.create_documents([texto_completo])
texts[:2]

[Document(metadata={}, page_content='HISTORIA DE COLOMBIA\n\nINTRODUCCIÓN\n\nLa historia de Colombia es un proceso complejo que se extiende desde los\nprimeros asentamientos humanos hasta la actualidad. El territorio\ncolombiano ha sido escenario de profundas transformaciones sociales,\npolíticas, económicas y culturales. Desde las civilizaciones indígenas\nhasta el Estado contemporáneo, Colombia ha construido una identidad\ndiversa y dinámica.\n\nPERÍODO PREHISPÁNICO\n\nMucho antes de la llegada de los europeos, el territorio estaba habitado\npor múltiples culturas indígenas. Entre las más destacadas se encuentran\nlos muiscas, taironas, quimbayas, zenúes y calimas. Estas sociedades\ndesarrollaron sistemas agrícolas avanzados, redes comerciales y\ncomplejas estructuras políticas.\n\nLos muiscas ocuparon el altiplano cundiboyacense y organizaron su\nsociedad en cacicazgos. La economía se basaba en el trueque, la sal y la\norfebrería. Los taironas construyeron asentamientos como Ciudad 

In [ ]:
len(texts[0].page_content)

986

In [ ]:
texts[1]

Document(metadata={}, page_content='CONQUISTA ESPAÑOLA\n\nEn 1499 comenzaron las exploraciones europeas en las costas del actual\nterritorio colombiano. Durante el siglo XVI se consolidó el dominio\nespañol. Se fundaron ciudades como Santa Marta, Cartagena y Santafé de\nBogotá. El proceso implicó violencia, explotación y transformación\ncultural profunda.\n\nVIRREINATO DE LA NUEVA GRANADA\n\nEn 1717 se creó el Virreinato de la Nueva Granada. Se consolidó una\nsociedad jerarquizada y profundamente religiosa. Las reformas borbónicas\ngeneraron tensiones económicas y sociales.\n\nINDEPENDENCIA\n\nEl 20 de julio de 1810 inició el proceso independentista. Tras varios\naños de conflicto, la Batalla de Boyacá en 1819 aseguró la victoria\npatriota. Se creó la Gran Colombia, que posteriormente se disolvió en\n1830.\n\nSIGLO XIX\n\nEl siglo XIX estuvo marcado por guerras civiles entre liberales y\nconservadores. En 1886 se promulgó una nueva Constitución centralista.\nLa Guerra de los Mil Días d

In [ ]:
len(texts[1].page_content)

997

Como se puede ver, usando `RecursiveCharacterTextSplitter`se respeta el tamaño máximo definido para los chuncks

## B. Document Splitter.

En pipelines reales de RAG, al usar **langChain** no se trabaja solo con texto plano, sino con `Document`.
Un Document Splitter:
* Recibe lista de `Document`
* Devuelve lista de `Document`
* Preserva metadatos

## 🔹 Ejemplo de Document Splitter

En esta sección se muestra cómo aplicar **Document Splitter** utilizando el archivo `Historia_Colombia.txt`.

A diferencia de `split_text()`, aquí trabajamos sobre objetos `Document`, lo que permite conservar metadatos como la fuente del documento.


In [ ]:
# Ejemplo de Document Splitter
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1️⃣ Leer archivo Historia_Colombia.txt
with open('Historia_Colombia.txt', 'r', encoding='utf-8') as f:
    texto_historia = f.read()

# 2️⃣ Crear objeto Document con metadata
doc = Document(
    page_content=texto_historia,
    metadata={'source': 'Historia_Colombia.txt'}
)

# 3️⃣ Crear splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# 4️⃣ Aplicar Document Splitter
docs_split = splitter.split_documents([doc])

# 5️⃣ Mostrar resultados
print(f'Total de fragmentos generados: {len(docs_split)}')
print('\nPrimer fragmento:')
print(docs_split[0].page_content[:500])
print('\nMetadata del primer fragmento:')
print(docs_split[0].metadata)


Total de fragmentos generados: 420

Primer fragmento:
HISTORIA DE COLOMBIA

INTRODUCCIÓN

La historia de Colombia es un proceso complejo que se extiende desde los
primeros asentamientos humanos hasta la actualidad. El territorio
colombiano ha sido escenario de profundas transformaciones sociales,
políticas, económicas y culturales. Desde las civilizaciones indígenas
hasta el Estado contemporáneo, Colombia ha construido una identidad
diversa y dinámica.

PERÍODO PREHISPÁNICO

Metadata del primer fragmento:
{'source': 'Historia_Colombia.txt'}


**NOTA:** En sistemas empresariales siempre usar `split_documents`.